In [1]:
import pandas as pd
import numpy as np

cust = pd.read_csv("C:/Users/pktal/Customer 360 Segmentation/customer_360_with_clusters.csv")
cust.head()

,customer_id,age,gender,income,occupation,city,tenure_years,avg_balance,monthly_deposits,monthly_withdrawals,...,high_credit_util,digital_engagement_score,txn_activity_score,total_investments,wealth_tier,risk_flag,risk_score,product_count,cluster_kmeans,cluster_gmm
0,1,59,Male,800000,Salaried,Danieltown,15,92515.92,1,9,...,1,0.368667,10,0.0,Low,0,2.0,1,2,2
1,2,63,Female,800000,Self-Employed,Littleberg,7,138097.40,13,16,...,0,0.634667,29,119949.0,Medium,0,0.0,3,2,3
2,3,39,Male,200000,Self-Employed,Markview,11,124329.40,14,8,...,1,0.517333,22,0.0,Low,0,2.0,2,1,0
3,4,44,Male,350000,Salaried,Lake Robertstad,8,156819.94,4,19,...,0,0.490667,23,0.0,Low,0,0.0,0,0,3
4,5,44,Female,800000,Salaried,Delgadoburgh,2,93515.51,9,13,...,0,0.534000,22,193258.0,Medium,0,0.0,3,0,3


In [2]:
cust["cluster_kmeans"].value_counts(normalize=True) * 100

0    49.000000
2    27.233333
1    23.766667
Name: cluster_kmeans, dtype: float64

In [3]:
# Selecting Key Business Metrics

profile_features = [
    "age","income","tenure_years",
    "avg_balance","digital_engagement_score",
    "spend_to_income","emi_to_income",
    "utilization_ratio","txn_activity_score",
    "total_investments","risk_score",
    "product_count"
]

cluster_profile = cust.groupby("cluster_kmeans")[profile_features].mean().round(2)
cluster_profile

,age,income,tenure_years,avg_balance,digital_engagement_score,spend_to_income,emi_to_income,utilization_ratio,txn_activity_score,total_investments,risk_score,product_count
cluster_kmeans,,,,,,,,,,,,
0,43.11,711054.42,10.15,81223.05,0.56,0.22,0.11,0.12,17.98,68276.60,0.06,0.99
1,41.53,291514.73,9.64,83260.36,0.56,0.48,0.83,0.28,17.60,125682.61,0.45,1.95
2,42.94,699877.60,10.49,82303.32,0.55,0.24,0.19,0.69,17.44,132648.86,2.13,2.01


In [4]:
# Product Penetration by Cluster

product_penetration = cust.groupby("cluster_kmeans")[
    ["has_credit_card","has_loan","has_investments"]
].mean().round(2)

product_penetration

,has_credit_card,has_loan,has_investments
cluster_kmeans,,,
0,0.35,0.33,0.30
1,0.65,0.79,0.50
2,0.98,0.51,0.53


In [5]:
# Adding Segment Labels

segment_map = {
    0: "Mass Retail / Low Engagement",
    1: "Credit Active Middle Segment",
    2: "Affluent & Investment Ready"
}

cust["segment_name"] = cust["cluster_kmeans"].map(segment_map)

In [6]:
persona_summary = cust.groupby("segment_name")[profile_features].mean().round(2)
persona_summary

,age,income,tenure_years,avg_balance,digital_engagement_score,spend_to_income,emi_to_income,utilization_ratio,txn_activity_score,total_investments,risk_score,product_count
segment_name,,,,,,,,,,,,
Affluent & Investment Ready,42.94,699877.60,10.49,82303.32,0.55,0.24,0.19,0.69,17.44,132648.86,2.13,2.01
Credit Active Middle Segment,41.53,291514.73,9.64,83260.36,0.56,0.48,0.83,0.28,17.60,125682.61,0.45,1.95
Mass Retail / Low Engagement,43.11,711054.42,10.15,81223.05,0.56,0.22,0.11,0.12,17.98,68276.60,0.06,0.99


In [7]:
# Product Gaps per Segment

cross_sell_matrix = cust.groupby("segment_name")[
    ["has_credit_card","has_loan","has_investments"]
].mean()

cross_sell_matrix = 1 - cross_sell_matrix
cross_sell_matrix.round(2)

,has_credit_card,has_loan,has_investments
segment_name,,,
Affluent & Investment Ready,0.02,0.49,0.47
Credit Active Middle Segment,0.35,0.21,0.50
Mass Retail / Low Engagement,0.65,0.67,0.70


In [8]:
# Business Logic Engine

def recommend_product(row):
    if row["segment_name"] == "Affluent & Investment Ready":
        if row["has_investments"] == 0:
            return "Wealth Products (MF, PMS, Insurance)"
        else:
            return "Premium Credit Card / Portfolio Upgrade"

    elif row["segment_name"] == "Credit Active Middle Segment":
        if row["has_credit_card"] == 0:
            return "Credit Card"
        elif row["has_loan"] == 0:
            return "Personal / Auto Loan"
        else:
            return "Limit Enhancement / Cross Loan"

    else:
        return "Savings + FD + Micro Insurance"


In [9]:
cust["cross_sell_offer"] = cust.apply(recommend_product, axis=1)

In [10]:
cust["cross_sell_offer"].value_counts()

Savings + FD + Micro Insurance             1470
Premium Credit Card / Portfolio Upgrade     431
Wealth Products (MF, PMS, Insurance)        386
Limit Enhancement / Cross Loan              360
Credit Card                                 248
Personal / Auto Loan                        105
Name: cross_sell_offer, dtype: int64

In [11]:
# Expected Conversion Assumptions

conversion_rate = {
    "Wealth Products (MF, PMS, Insurance)": 0.08,
    "Premium Credit Card / Portfolio Upgrade": 0.06,
    "Credit Card": 0.10,
    "Personal / Auto Loan": 0.07,
    "Limit Enhancement / Cross Loan": 0.05,
    "Savings + FD + Micro Insurance": 0.04
}

avg_revenue = {
    "Wealth Products (MF, PMS, Insurance)": 15000,
    "Premium Credit Card / Portfolio Upgrade": 8000,
    "Credit Card": 5000,
    "Personal / Auto Loan": 12000,
    "Limit Enhancement / Cross Loan": 6000,
    "Savings + FD + Micro Insurance": 3000
}

In [12]:
cust["expected_revenue"] = cust["cross_sell_offer"].map(avg_revenue) * \
                           cust["cross_sell_offer"].map(conversion_rate)

cust["expected_revenue"].sum()

1166680.0

In [14]:
cust.to_csv("C:/Users/pktal/Customer 360 Segmentation/customer_360_final_with_strategy.csv", index=False)

cluster_profile.to_csv("C:/Users/pktal/Customer 360 Segmentation/cluster_summary.csv")
persona_summary.to_csv("C:/Users/pktal/Customer 360 Segmentation/persona_summary.csv")
cross_sell_matrix.to_csv("C:/Users/pktal/Customer 360 Segmentation/cross_sell_opportunity_matrix.csv")

print("Cluster profiling and strategy outputs saved.")

Cluster profiling and strategy outputs saved.
